<a href="https://colab.research.google.com/github/seulkiki/TFMaster/blob/master/tf04_Save_and_load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 https://www.tensorflow.org/tutorials/keras/save_and_load텍스트*

_  checkpoint를 어떻게 활용할 것인가?

In [ ]:
pip install pyyaml h5py #HDF5 포맷으로 모델을 저장하기 위해서 필요

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0




In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance(모델객체 만들기)
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 훈련과정을 이어서 함
#Checkpoint callback
# 훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것이 많이 사용하는 방법입니다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다. tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

In [ ]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)


In [ ]:
# Create a callback that saves the model's weights(모델의 가중치를 저장하는 콜백 만들기)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=2)


In [ ]:
# Train the model with the new callback(새로운 콜백으로 모델 훈련)
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training(훈련에 전달하기 위해서 쓰는 코드)


Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 1.1646 - sparse_categorical_accuracy: 0.6520 - val_loss: 0.7508 - val_sparse_categorical_accuracy: 0.7630

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 0.4334 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5442 - val_sparse_categorical_accuracy: 0.8320

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2845 - sparse_categorical_accuracy: 0.9300 - val_loss: 0.4793 - val_sparse_categorical_accuracy: 0.8580

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2150 - sparse_categorical_accuracy: 0.9450 - val_loss: 0.4668 - val_sparse_categorical_accuracy: 0.8540

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1543 - sparse_cat

In [ ]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

#### 훈련하지 않은 새로운 모델을 만들기

In [ ]:
model = create_model()#기본 모델 객체 만들기

In [ ]:
# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

#모델의 성능: 얼마나 무작위로 모델을 갖고 오는지

32/32 - 0s - loss: 2.3200 - sparse_categorical_accuracy: 0.0930
Untrained model, accuracy:  9.30%


In [ ]:
## 체크포인트에서 가중치를 로드하고 다시 평가

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

In [ ]:
# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4504 - sparse_categorical_accuracy: 0.8550
Restored model, accuracy: 85.50%


## 체크포인트 콜백 매개변수

체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath= checkpoint_path,
                                                 verbose = 1,
                                                 save_weights_only = True,
                                                 save_freq = 5*batch_size)

In [ ]:
model = create_model()

In [ ]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
os.listdir(checkpoint_dir)


['cp-0005.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0045.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0050.ckpt.index']

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

### 노트: 텐서플로는 기본적으로 최근 5개의 체크포인트만 저장

In [ ]:
model = create_model()

model.load_weights(latest)

In [ ]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4926 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%


In [ ]:
pwd

'/content'

## save_weights

In [ ]:
model.save_weights('/checkpoints/my_checkpoint') #가중치 저장

In [ ]:
model = create_model()#새로운 모델 객체 만들기

In [ ]:
model.load_weights('/checkpoints/my_checkpoint')# 가중치복원

In [ ]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)# 모델 평가
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4926 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%


In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1339 - sparse_categorical_accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4154 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2875 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2219 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1480 - sparse_categorical_accuracy: 0.9750


In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
ls saved_model#saved에 my_model이라는 디렉토리가 있음

ls: cannot access 'saved_model#saved에': No such file or directory
ls: cannot access 'my_model이라는': No such file or directory
ls: cannot access '디렉토리가': No such file or directory
ls: cannot access '있음': No such file or directory


In [ ]:
ls saved_model/my_model

assets/  keras_metadata.pb  saved_model.pb  variables/


In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4279 - sparse_categorical_accuracy: 0.8590
Restored model, accuracy: 85.90%
(1000, 10)


## HDF5 format******(많이 사용)

In [ ]:
model = create_model() #모델 인스턴스 만들기
model.fit(train_images, train_labels, epochs=5)


# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')


Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1444 - sparse_categorical_accuracy: 0.6900
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4370 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2859 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2139 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1625 - sparse_categorical_accuracy: 0.9560


In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')# 확장자 정하기

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4206 - sparse_categorical_accuracy: 0.8710
Restored model, accuracy: 87.10%


###### 참고할 것: 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"


HDF5 포맷은 대용량 데이터를 처리하기 위한 파일 형식으로, 많은 양의 데이터를 안정적으로 저장할 뿐만 아니라, Random Access 및 빠른 검색 등을 지원